In [6]:
import torch
import torchaudio
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    BitsAndBytesConfig
)
from peft import PeftModel
from redlines import Redlines

# === Config ===
base_model_name = "openai/whisper-large-v2"
adapter_checkpoint = "/mnt/sda1/FYP_2024/Helitha/DatasetsZ/asr_sinhala/temp/checkpoint-20244"
audio_file = "/mnt/sda1/FYP_2024/Helitha/DatasetsZ/asr_sinhala/Wakkada_Langa_Diya-Kalu_W_D_Amaradewa_Sarigama_lk-AudioTrimmer.com.mp3"

# === Load Processor ===
print("🔧 Loading processor...")
processor = WhisperProcessor.from_pretrained(base_model_name)

# === Load Base Model (No Adapter) ===
print("🧠 Loading *pure* base model (8-bit)...")
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

base_model = WhisperForConditionalGeneration.from_pretrained(
    base_model_name,
    device_map="auto",
    quantization_config=bnb_config
)
base_model.config.forced_decoder_ids = None
base_model.eval()

# === Load Another Base Model for Fine-Tuning ===
print("🧪 Preparing second model instance for fine-tuning...")
adapter_base_model = WhisperForConditionalGeneration.from_pretrained(
    base_model_name,
    device_map="auto",
    quantization_config=bnb_config
)
adapter_base_model.config.forced_decoder_ids = None
adapter_base_model.eval()

print("🔌 Loading LoRA adapter into second model...")
fine_tuned_model = PeftModel.from_pretrained(adapter_base_model, adapter_checkpoint)
fine_tuned_model.config.forced_decoder_ids = None
fine_tuned_model.eval()

# === Transcription Function ===
def transcribe(model, audio_path, chunk_duration=30):
    waveform, sr = torchaudio.load(audio_path)

    if sr != 16000:
        waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)

    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    total_samples = waveform.shape[1]
    chunk_samples = chunk_duration * 16000

    all_text = []
    for start in range(0, total_samples, chunk_samples):
        end = min(start + chunk_samples, total_samples)
        chunk = waveform[:, start:end]

        input_features = processor.feature_extractor(
            chunk.squeeze().numpy(), sampling_rate=16000, return_tensors="pt"
        ).input_features.to(model.device).to(dtype=torch.float16)

        attention_mask = torch.ones(input_features.shape[:2], dtype=torch.long).to(model.device)

        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                attention_mask=attention_mask,
                language="si",
                task="transcribe"
            )

        text = processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        all_text.append(text)

    return " ".join(all_text)

# === Main Execution ===
if __name__ == "__main__":
    print("🎧 Transcribing with base model...")
    base_output = transcribe(base_model, audio_file)

    print("🧪 Transcribing with fine-tuned model...")
    tuned_output = transcribe(fine_tuned_model, audio_file)

    print("\n📝 Base Model Output:\n", base_output)
    print("\n📝 Fine-Tuned Model Output:\n", tuned_output)

    print("\n📊 Comparing with Redlines...")
    diff = Redlines(base_output, tuned_output)
    #print(diff.output_markdown)
    from IPython.display import Markdown, display
    display(Markdown(diff.output_markdown))



🔧 Loading processor...
🧠 Loading *pure* base model (8-bit)...
🧪 Preparing second model instance for fine-tuning...
🔌 Loading LoRA adapter into second model...
🎧 Transcribing with base model...
🧪 Transcribing with fine-tuned model...

📝 Base Model Output:
  මෙම්ල්ල් පොත්තියි මෙම්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්  මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තා  මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙ  ම

<span style='color:red;font-weight:700;text-decoration:line-through;'>මෙම්ල්ල් පොත්තියි මෙම්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්ල්  මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තාරයි මින්තා  මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙන්න මෙ  මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල් මෙම්ල්  අපි දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින දින</span><span style='color:green;font-weight:700;'>වක්කර ළඟ සීර් වැටෙන තාවයට චිස්වර අහස්සේ වලාකුලින් බිදුලිය කෙතුවා. වැටවට කෙරුවා.තුන් දවසක් අමනාපෙන් සිටි කලු නෝකාඩු බැල්මෙන් වැලුවා. ගංග උතුරා හිත පිරෙන්න ගලමතුපිට මල පිපෙන්න මලවට බමරින්දු නටන් වැහි වැහැලා ගඟ පිරෙන්න. වගේ උන්ග හිනා වෙලන් කළු හිනාවෙනක්. ගිනාවේ අනුකළු හිනාවෙයන් කළු හිනාවෙයන්</span>